In [10]:
import pygame
import sys
from datetime import datetime
# Initialize Pygame
pygame.init()

# Set up the screen
WIDTH, HEIGHT = 400, 200
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Traffic Lights")

# Define colors
RED = (255, 0, 0)
GREEN = (0, 255, 0)
GRAY = (100, 100, 100)

# Traffic light positions and sizes
LIGHT_WIDTH, LIGHT_HEIGHT = 40, 120
MARGIN = 20
LIGHT_GAP = (WIDTH - 2 * MARGIN - 4 * LIGHT_WIDTH) / 3
LIGHT_Y = HEIGHT // 2 - LIGHT_HEIGHT // 2

# Initialize light states
light_states = [0, 0, 0, 0]  # 0 for red, 1 for green

current_datetime = datetime.now()
current_data = [1,current_datetime.year, current_datetime.month, current_datetime.day, current_datetime.hour, current_datetime.weekday()]

from get_vehicles import get_number_of_vehicles

arr = get_number_of_vehicles(current_data)
print(arr)
mean_time = 100
clock = pygame.time.Clock()
# Main loop
# light_states = [0, 0, 0, 0]
light_index = 0
while True:
    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            if pygame.K_1 <= event.key <= pygame.K_4:
                light_index = event.key - pygame.K_1
                light_states[light_index] = 1 - light_states[light_index]

    # Clear the screen
    screen.fill((255, 255, 255))
    light_states[light_index] = 1
    

    for i, state in enumerate(light_states):
        light_color = GREEN if state else RED
        pygame.draw.rect(screen, GRAY, (MARGIN + i * (LIGHT_WIDTH + LIGHT_GAP), LIGHT_Y, LIGHT_WIDTH, LIGHT_HEIGHT))
        pygame.draw.circle(screen, light_color,
                           (MARGIN + i * (LIGHT_WIDTH + LIGHT_GAP) + LIGHT_WIDTH // 2, LIGHT_Y + LIGHT_HEIGHT // 4), 20)
        # pygame.draw.circle(screen, RED,
        #                    (MARGIN + i * (LIGHT_WIDTH + LIGHT_GAP) + LIGHT_WIDTH // 2, LIGHT_Y + LIGHT_HEIGHT // 2), 20)
        # pygame.draw.circle(screen, RED,
        #                    (MARGIN + i * (LIGHT_WIDTH + LIGHT_GAP) + LIGHT_WIDTH // 2, LIGHT_Y + 3 * LIGHT_HEIGHT // 4),
        #                    20)
    pygame.time.wait(int(mean_time * arr[light_index]))

    # Update the display
    pygame.display.flip()
    light_states[light_index] = 0
    light_index = (light_index+1)%(len(light_states))


{'option': '1', 'data': [1, 2024, 4, 6, 15, 5]}
[45.62532424926758, 14.787566184997559, 14.052435874938965, 7.437169551849365]


SystemExit: 

In [ ]:
import numpy as np
import argparse
import time
import cv2
import os
'''
# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True,
	help="path to input image")
ap.add_argument("-y", "--yolo", required=True,
	help="base path to YOLO directory")
ap.add_argument("-c", "--confidence", type=float, default=0.5,
	help="minimum probability to filter weak detections")
ap.add_argument("-t", "--threshold", type=float, default=0.3,
	help="threshold when applyong non-maxima suppression")
args = vars(ap.parse_args())
'''
# load the COCO class labels our YOLO model was trained on
labelsPath = ("coco.names")
LABELS = open(labelsPath).read().strip().split("\n")
#print(LABELS)

# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")

# derive the paths to the YOLO weights and model configuration
modelConfiguration = "yolov3.cfg";
modelWeights = "yolov3.weights";
 
def getOutputsNames(net):
# Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i - 1] for i in net.getUnconnectedOutLayers()]

net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [ ]:
def get_queue_length(image):
# image = np.array(data)
# image = image.astype('uint8')

    image = ((image / np.max(image)) * 255).astype('uint8')

    # (H, W) = image.shape[:2]
    # print("The iiumage shape is:",image.shape)
    # determine only the *output* layer names that we need from YOLO
    blob = cv2.dnn.blobFromImage(image, size = (416,416), mean = (0,0,0), swapRB=False, crop=False)

    # Sets the input to the network
    net.setInput(blob)

    # Runs the forward pass to get output of the output layers
    outs = net.forward(getOutputsNames(net))
    # end = time.time()
    #print("The layers are:",layerOutputs)

    # show timing information on YOLO
    # print("[INFO] YOLO took {:.6f} seconds".format(end - start))

    # initialize our lists of detected bounding boxes, confidences, and
    # class IDs, respectively
    boxes = []
    confidences = []
    classIDs = []

    # loop over each of the layer outputs
    for output in outs:
    # loop over each of the detections
        for detection in output:
            #print(detection)
            # extract the class ID and confidence (i.e., probability) of
            # the current object detection
            
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence!=0:
                print(confidence)

            bicycle_cnt = 0
            vehicle_cnt = 0
            # filter out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > 0.40 and classID == 4:
                bicycle_cnt = bicycle_cnt + 1
            if confidence > 0.40 and classID == 2:
                vehicle_cnt = vehicle_cnt+ 1
    return vehicle_cnt

In [7]:
import pygame
import sys

# Initialize pygame
pygame.init()


# Set up some constants
LIGHT_WIDTH = 50
LIGHT_HEIGHT = 150
LIGHT_GAP = 10
MARGIN = 20
LIGHT_Y = 50
GREEN = (0, 255, 0)
RED = (255, 0, 0)
GRAY = (200, 200, 200)

# Create the screen
screen = pygame.display.set_mode((800, 600))

# Initialize light states
light_states = [0, 0, 0, 0]

from datetime import datetime
current_datetime = datetime.now()
current_data = [1,current_datetime.year, current_datetime.month, current_datetime.day, current_datetime.hour, current_datetime.weekday()]

from get_vehicles import get_number_of_vehicles

arr = get_number_of_vehicles(current_data)

# Initialize the clock
clock = pygame.time.Clock()

# Initialize the light index
light_index = 0

mean_time = 100

while True:
    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    # Clear the screen
    screen.fill((255, 255, 255))

    # Update the light states
    light_states = [0, 0, 0, 0]
    light_states[light_index] = 1

    # Draw the lights
    for i, state in enumerate(light_states):
        light_color = GREEN if state else RED
        pygame.draw.rect(screen, GRAY, (MARGIN + i * (LIGHT_WIDTH + LIGHT_GAP), LIGHT_Y, LIGHT_WIDTH, LIGHT_HEIGHT))
        pygame.draw.circle(screen, light_color,
                           (MARGIN + i * (LIGHT_WIDTH + LIGHT_GAP) + LIGHT_WIDTH // 2, LIGHT_Y + LIGHT_HEIGHT // 4), 20)
        # pygame.draw.circle(screen, RED,
        #                    (MARGIN + i * (LIGHT_WIDTH + LIGHT_GAP) + LIGHT_WIDTH // 2, LIGHT_Y + LIGHT_HEIGHT // 2), 20)
        # pygame.draw.circle(screen, RED,
        #                    (MARGIN + i * (LIGHT_WIDTH + LIGHT_GAP) + LIGHT_WIDTH // 2, LIGHT_Y + 3 * LIGHT_HEIGHT // 4),
        #                    20)

    # Update the display
    pygame.display.flip()
    image_now = cv2.imread("image.jpg")
    # Implement many images
    queue_length = get_queue_length(image_now)
    # Wait for a second
    pygame.time.wait(int(mean_time * arr[light_index]*queue_length))

    # Update the light index
    light_index = (light_index + 1) % len(light_states)

{'option': '1', 'data': [1, 2024, 4, 6, 2, 5]}
Error: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022CCF1E9FD0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
Error: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022CCF1E86D0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
Error: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022CCF35AB50>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it

SystemExit: 